# Iterando registros de tamano fijo

5.8

- Problema

En lugar de iterar sobre un archivo por lineas, desea iterar sobre registros de tamano fijo o bloques.

- Solucion

Use `iter()` con un invocable y un valor centinela, junto con `functools.partial()`.

```python
RECORD_SIZE = 32
with open('somefile.data', 'rb') as f:
    registros = iter(partial(f.read, RECORD_SIZE), b'')
    for r in registros:
        ...
```

El objeto `registros` es un iterable que produce bloques de tamano fijo hasta llegar al final del archivo.

Tenga en cuenta que el ultimo bloque puede tener menos bytes si el tamano del archivo no es un multiplo exacto del tamano del registro.

Una caracteristica poco conocida de `iter()` es que puede crear un iterador si le pasa un invocable y un valor centinela. El iterador llama al invocable una y otra vez hasta que devuelve el centinela, momento en el que se detiene la iteracion.

En la solucion, `functools.partial` se usa para crear un invocable que lee un numero fijo de bytes del archivo cada vez que se llama. El centinela `b''` es lo que se devuelve cuando se llega al final del archivo.

Por ultimo, la solucion muestra el archivo en modo binario. Para registros de tamano fijo suele ser lo mas comun.

En archivos de texto, leer linea por linea (el comportamiento de iteracion predeterminado) es mas habitual.

In [1]:
from functools import partial

In [2]:
RECORD_SIZE = 100
with open ('somefile.data', 'rb') as f:
    registros = iter (partial (f.read, RECORD_SIZE), b"")
    for r in registros:
        print(r)

b'functools \xe2\x80\x94 Funciones de orden superior y operaciones sobre objetos invocables\nC\xc3\xb3digo fuente: Lib'
b'/functools.py\n\nEl m\xc3\xb3dulo functools es para funciones de orden superior: funciones que act\xc3\xbaan o ret'
b'ornan otras funciones. En general, cualquier objeto invocable puede ser tratado como una funci\xc3\xb3n pa'
b'ra los prop\xc3\xb3sitos de este m\xc3\xb3dulo.\n\nEl m\xc3\xb3dulo functools define las siguientes funciones:\n\n@functoo'
b'ls.cached_property(func)\nTransforma un m\xc3\xa9todo de una clase en una propiedad cuyo valor se computa u'
b'na vez y luego se almacena como un atributo normal durante la vida de la instancia. Similar a proper'
b'ty(), con la adici\xc3\xb3n de caching. \xc3\x9atil para propiedades calculadas costosas de instancias que de ot'
b'ra manera son efectivamente inmutables.\n\nEjemplo:\n\nclass DataSet:\n    def __init__(self, sequence_of'
b'_numbers):\n        self._data = sequence_of_numbers\n\n    @cached_property\n    de

# Leer datos binarios en un buffer mutable

5.9

- Problema

Quiere leer datos binarios directamente en un buffer mutable, sin un proceso intermedio de copias. Quizas desee mutar los datos en el lugar y volver a escribirlos en un archivo.

- Solucion

Para leer datos en un arreglo mutable, use el metodo `readinto()` del archivo.

Por ejemplo:

In [3]:
import os.path

In [4]:
def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

In [5]:
with open('sample.bin', 'wb') as f:
    f.write(b'Hello World')

In [6]:
buf = read_into_buffer('sample.bin')

In [7]:
buf

bytearray(b'Hello World')

In [8]:
buf[0:5] = b'Hallo'

In [9]:
buf

bytearray(b'Hallo World')

In [10]:
with open('newsample.bin', 'wb') as f:
    f.write(buf)

El metodo `readinto()` se puede usar para llenar cualquier arreglo preasignado con datos. Esto incluye arreglos del modulo `array` o librerias como `numpy`. A diferencia de `read()`, `readinto()` llena un buffer existente en lugar de asignar nuevos objetos y devolverlos. Por lo tanto, puede reducir asignaciones de memoria.

Por ejemplo, si esta leyendo un archivo binario que consta de registros de igual tamano, puede escribir codigo como este:

In [11]:
record_size = 16
# Tamaño de cada registro (ajustar valor)
buf = bytearray(record_size)
with open('somefile', 'rb') as f:
    while True:
        n = f.readinto(buf)
        if n < record_size:
            break
        # Usa el contenido de buf

In [12]:
buf

bytearray(b'Hello\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [13]:
m1 = memoryview(buf)

In [14]:
m2 = m1[-5:]
m2

In [15]:
m2[:] = b'WORLD'

In [16]:
print(buf)

bytearray(b'Hello\r\n\x00\x00\x00\x00WORLD')


Una precaucion al usar `f.readinto()` es que siempre debe verificar su codigo de retorno, que es el numero de bytes realmente leidos.
Si el numero de bytes es menor que el tamano del buffer proporcionado, podria indicar datos truncados o corruptos (por ejemplo, si esperaba leer un numero exacto de bytes).
Finalmente, este atento a otras funciones relacionadas con `into` en varios modulos de la biblioteca (por ejemplo, `recv_into()`, `pack_into()`, etc.). Muchas otras partes de Python tienen soporte para E/S directa o acceso a datos que se pueden usar para completar o alterar el contenido de arreglos y buffers.
Consulte la Receta 6.12 para ver un ejemplo mas avanzado de interpretacion de estructuras binarias y uso de vistas de memoria.

# Archivos binarios con asignacion de memoria

5.10

- Problema

Desea mapear en memoria un archivo binario en una matriz de bytes mutable, posiblemente para acceder a su contenido o realizar modificaciones in situ.

- Solucion

Utilice el modulo `mmap` para mapear archivos. Aqui hay una funcion de utilidad que muestra como abrir un archivo y crear un mapa de memoria de forma portable:

In [17]:
import os
import mmap

In [18]:
def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    try:
        return mmap.mmap(fd, size, access=access)
    finally:
        os.close(fd)

Para usar esta funcion, necesita tener un archivo ya creado y lleno de datos.
A continuacion, se muestra un ejemplo de como podria crear inicialmente un archivo y expandirlo en disco:

In [19]:
size = 1_000_000
with open('data', 'wb') as f:
    f.truncate(size)

In [20]:
m = memory_map('data')
len(m)

1000000

In [21]:
type(m)

mmap.mmap

In [22]:
m[0:10]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [23]:
m[0]

0

In [24]:
m[0:11] = b'Hello World'

In [25]:
with open('data', 'rb') as f:
    print(f.read(11))

b'Hello World'


El objeto `mmap` devuelto por `mmap()` tambien se puede utilizar como administrador de contexto, en cuyo caso el recurso se cierra automaticamente.
Por ejemplo:

In [26]:
with memory_map('data') as m:
    print(len(m))
    print(m[0:10])

1000000
b'Hello Worl'


In [27]:
m.closed

True

De forma predeterminada, la funcion `memory_map()` que se muestra abre un archivo tanto para lectura como para escritura.
Cualquier modificacion realizada a los datos se copia al archivo original. Si necesita acceso de solo lectura, proporcione `mmap.ACCESS_READ` en el argumento `access`.
Por ejemplo:
```python
m = memory_map(filename, access=mmap.ACCESS_READ)
```

Si tiene la intencion de modificar los datos localmente, pero no desea que esos cambios se vuelvan a escribir en el archivo original, use `mmap.ACCESS_COPY`:
```python
m = memory_map(filename, access=mmap.ACCESS_COPY)
```

El uso de `mmap` para mapear archivos en memoria puede ser un medio eficiente y elegante para acceder al contenido de un archivo. En lugar de abrir un archivo y realizar varias combinaciones de llamadas `seek()`, `read()` y `write()`, puede mapear el archivo y acceder a los datos mediante operaciones de corte.

Normalmente, la memoria expuesta por `mmap()` parece un objeto `bytearray`. Sin embargo, puede interpretar los datos de manera diferente usando una vista de memoria.
Por ejemplo:

In [28]:
m = memory_map('data')
# Memoryview of unsigned integers
v = memoryview(m).cast('I')
v[0] = 7
m[0:4]

b'\x07\x00\x00\x00'

In [29]:
m[0:4] = b'\x07\x01\x00\x00'
v[0]

263

Debe enfatizarse que el mapeo de memoria de un archivo no hace que todo el archivo se lea en memoria. Es decir, no se copia a ningun tipo de arreglo o buffer.
En cambio, el sistema operativo reserva una seccion de memoria virtual para el contenido del archivo. A medida que accede a diferentes regiones, esas partes del archivo se leen y se mapean en la region de memoria segun sea necesario.
Las partes del archivo a las que nunca se accede permanecen en el disco. Todo esto sucede de forma transparente.

Si mas de un interprete de Python asigna memoria al mismo archivo, el objeto `mmap` resultante se puede utilizar para intercambiar datos entre interpretes. Es decir, todos los interpretes pueden leer/escribir datos simultaneamente, y los cambios realizados en un interprete aparecen automaticamente en los demas.
Obviamente, se requiere cuidado adicional para sincronizar, pero este enfoque se usa a veces como alternativa a transmitir datos mediante tuberias o sockets.

Como se muestra, esta receta se ha escrito para ser lo mas general posible, funcionando tanto en Unix como en Windows. Tenga en cuenta que existen algunas diferencias de plataforma en el uso de la llamada `mmap()` bajo el capot. Ademas, existen opciones para crear regiones de memoria mapeadas de forma anonima. Si le interesa, lea la documentacion de Python sobre el tema.

# Manipulacion de nombres de ruta

5.11

- Problema

Debe manipular nombres de rutas para encontrar el nombre base del archivo, el nombre del directorio, la ruta absoluta y asi sucesivamente.

- Solucion

Para manipular nombres de ruta, use las funciones del modulo `os.path`. A continuacion, un ejemplo interactivo que ilustra algunas caracteristicas clave:

In [30]:
import os

In [31]:
path = "/etc/jupyter/nbconfig/edit.json"

In [32]:
# Obtén el último componente de la ruta
os.path.basename(path)

'edit.json'

In [33]:
# Obtener el nombre del directorio
os.path.dirname(path)

'/etc/jupyter/nbconfig'

In [34]:
os.path.join('etc', 'jupyter', "nbconfig", os.path.basename(path))

'etc\\jupyter\\nbconfig\\edit.json'

In [38]:
# Expand the user's home directory
os.path.expanduser(path)

'/etc/jupyter/nbconfig/edit.json'

In [39]:
os.path.splitext(path)

('/etc/jupyter/nbconfig/edit', '.json')

Para cualquier manipulacion de nombres de archivo, debe usar `os.path` en lugar de crear su propio codigo con operaciones de cadena. En parte, esto es por portabilidad. `os.path` conoce las diferencias entre Unix y Windows y puede manejar de manera confiable nombres de archivo como `Data/data.csv` y `Data\data.csv`.
Segundo, no deberia perder tiempo reinventando la rueda. Por lo general, es mejor utilizar la funcionalidad que ya se le proporciona.

# Prueba de la existencia de un archivo

5.12

- Problema

Necesita probar si existe o no un archivo o directorio.

- Solucion

Utilice el modulo `os.path` para probar la existencia de un archivo o directorio.

Por ejemplo:

In [25]:
os.path.exists("somefile")

True

In [40]:
os.path.exists('/tmp/spam')

False

Puede realizar mas pruebas para ver que tipo de archivo podria ser.
Estas pruebas regresan `False` si el archivo en cuestion no existe:

In [44]:
os.path.isfile('somefile.bin')

True

In [45]:
os.path.isdir('/etc/passwd')

False

In [43]:
os.path.realpath('/usr/local/bin/python3')

'D:\\usr\\local\\bin\\python3'

Si necesita obtener metadatos (por ejemplo, el tamano del archivo o la fecha de modificacion), tambien estan disponibles en el modulo `os.path`.

In [10]:
os.path.getsize('/etc/passwd')

2568

In [11]:
os.path.getmtime('/etc/passwd')

1608601640.2807653

In [12]:
import time
time.ctime(os.path.getmtime('/etc/passwd'))

'Mon Dec 21 22:47:20 2020'

La prueba de archivos es una operacion sencilla con `os.path`. Al escribir scripts, es posible que deba preocuparse por los permisos, especialmente para operaciones que obtienen metadatos.
Por ejemplo:
```python
>>> os.path.getsize('/Users/guido/Desktop/foo.txt')
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
File "/usr/local/lib/python3.3/genericpath.py", line 49, in getsize
return os.stat(filename).st_size
PermissionError: [Errno 13] Permission denied: '/Users/guido/Desktop/foo.txt'
```

# Obtener una lista de directorios

5.13

- Problema

Desea obtener una lista de los archivos contenidos en un directorio del sistema de archivos.

- Solucion

Utilice la funcion `os.listdir()` para obtener una lista de archivos en un directorio:

In [46]:
import os

In [47]:
actual = os.getcwd()
actual

'd:\\recetas-python\\Capitulo 5 (Archivos u I'

In [48]:
os.listdir(actual)

['cap5-1-7.ipynb',
 'cap5_part_02.ipynb',
 'data',
 'data.bin',
 'ejemplo.txt',
 'ejemplo_pathlib.txt',
 'newsample.bin',
 'readme.txt',
 'saludo.txt',
 'saludo2.txt',
 'sample.bin',
 'somefile',
 'somefile.bin',
 'somefile.data',
 'somefile.txt',
 'somefile2.bin',
 'somefile3',
 'somefile4']

Esto le dara la lista de entradas en bruto, incluidos archivos, subdirectorios, enlaces simbolicos, etc. Si necesita filtrar los datos de alguna manera, considere usar una comprension de lista combinada con varias funciones en la biblioteca `os.path`.
Por ejemplo:

In [49]:
archivos = [arch for arch in os.listdir(actual)
            if os.path.isfile(os.path.join(actual, arch))]

In [50]:
archivos

['cap5-1-7.ipynb',
 'cap5_part_02.ipynb',
 'data',
 'data.bin',
 'ejemplo.txt',
 'ejemplo_pathlib.txt',
 'newsample.bin',
 'readme.txt',
 'saludo.txt',
 'saludo2.txt',
 'sample.bin',
 'somefile',
 'somefile.bin',
 'somefile.data',
 'somefile.txt',
 'somefile2.bin',
 'somefile3',
 'somefile4']

In [51]:
carpetas = [arch for arch in os.listdir(actual)
            if os.path.isdir(os.path.join(actual, arch))]

In [52]:
carpetas

[]

Los metodos `startswith()` y `endswith()` de las cadenas tambien pueden ser utiles para filtrar el contenido de un directorio.
Por ejemplo:

In [53]:
notebook = [name for name in os.listdir(actual) if name.endswith('.ipynb')]

In [54]:
notebook

['cap5-1-7.ipynb', 'cap5_part_02.ipynb']

Para la coincidencia de nombres de archivo, es posible que desee utilizar los modulos `glob` o `fnmatch` en su lugar.
Por ejemplo:

In [55]:
import glob
binarios = glob.glob(actual+'/*.bin')

In [56]:
binarios

['d:\\recetas-python\\Capitulo 5 (Archivos u I\\data.bin',
 'd:\\recetas-python\\Capitulo 5 (Archivos u I\\newsample.bin',
 'd:\\recetas-python\\Capitulo 5 (Archivos u I\\sample.bin',
 'd:\\recetas-python\\Capitulo 5 (Archivos u I\\somefile.bin',
 'd:\\recetas-python\\Capitulo 5 (Archivos u I\\somefile2.bin']

In [57]:
from fnmatch import fnmatch
binarios2 = [name for name in os.listdir(actual) if fnmatch(name, '*.bin')]

In [58]:
binarios2

['data.bin', 'newsample.bin', 'sample.bin', 'somefile.bin', 'somefile2.bin']

In [39]:
glob.glob(actual+'/*lu*.*')

['/home/emi/Escritorio/Libro recetas/cap 5/saludo.txt']

In [59]:
[name for name in os.listdir(actual) if fnmatch(name, '*me*')]

['readme.txt',
 'somefile',
 'somefile.bin',
 'somefile.data',
 'somefile.txt',
 'somefile2.bin',
 'somefile3',
 'somefile4']

Obtener una lista de directorio es facil, pero solo le brinda los nombres de las entradas. Si desea metadatos adicionales, como tamanos de archivo y fechas de modificacion, use funciones adicionales en `os.path` o `os.stat()` para recopilar los datos.
Por ejemplo:

In [60]:
import os
import glob
import time

In [61]:
actual   = os.getcwd()
binarios = glob.glob(actual+'/*.bin')

In [63]:
metadatos = [ (name ,os.path.getsize(name), time.ctime(os.path.getmtime(name)))
                for name in binarios]

In [64]:
metadatos

[('d:\\recetas-python\\Capitulo 5 (Archivos u I\\data.bin',
  28,
  'Fri Feb  6 21:39:01 2026'),
 ('d:\\recetas-python\\Capitulo 5 (Archivos u I\\newsample.bin',
  11,
  'Fri Feb  6 23:57:29 2026'),
 ('d:\\recetas-python\\Capitulo 5 (Archivos u I\\sample.bin',
  11,
  'Fri Feb  6 23:57:20 2026'),
 ('d:\\recetas-python\\Capitulo 5 (Archivos u I\\somefile.bin',
  11,
  'Fri Feb  6 21:37:20 2026'),
 ('d:\\recetas-python\\Capitulo 5 (Archivos u I\\somefile2.bin',
  10,
  'Fri Feb  6 21:38:52 2026')]

In [65]:
file_metadata = [(name, os.stat(name)) for name in binarios]
for name, meta in file_metadata:
    print(name, meta.st_size, time.ctime(meta.st_mtime))

d:\recetas-python\Capitulo 5 (Archivos u I\data.bin 28 Fri Feb  6 21:39:01 2026
d:\recetas-python\Capitulo 5 (Archivos u I\newsample.bin 11 Fri Feb  6 23:57:29 2026
d:\recetas-python\Capitulo 5 (Archivos u I\sample.bin 11 Fri Feb  6 23:57:20 2026
d:\recetas-python\Capitulo 5 (Archivos u I\somefile.bin 11 Fri Feb  6 21:37:20 2026
d:\recetas-python\Capitulo 5 (Archivos u I\somefile2.bin 10 Fri Feb  6 21:38:52 2026


Por ultimo, tenga en cuenta que existen problemas sutiles que pueden surgir al manejar nombres de archivo relacionados con codificaciones. Normalmente, las entradas devueltas por una funcion como `os.listdir()` se decodifican de acuerdo con la codificacion de nombre de archivo predeterminada del sistema. Sin embargo, es posible encontrar nombres de archivo no decodificables en ciertas circunstancias.

# Anexo: alternativas modernas (stdlib)

- `pathlib.Path` para manipular rutas con metodos como `exists()`, `is_file()`, `is_dir()`, `stat()`, `read_text()`, `read_bytes()`, `write_text()` y `write_bytes()`.
- `Path.iterdir()`, `Path.glob()` y `Path.rglob()` para listar y filtrar entradas de directorio.
- `os.scandir()` cuando necesite rendimiento y metadatos en listados grandes.
- `pathlib.PurePath` para trabajar con rutas sin tocar el sistema de archivos (por ejemplo, pruebas o rutas remotas).
- `itertools.batched()` (Python 3.12+) si necesita trocear iterables en bloques sin cargar todo en memoria.

Ejemplo 1. Uso basico de `pathlib.Path` para escribir, leer y borrar un archivo de texto. La idea es reemplazar patrones con `open()` por metodos mas expresivos cuando se trabaja con rutas.

In [ ]:
from pathlib import Path

# Crea un objeto Path para el archivo demo_pathlib.txt
demo = Path("demo_pathlib.txt")

# Escribe el texto "hola\n" en el archivo demo_pathlib.txt usando UTF-8 como codificación
demo.write_text("hola\n", encoding="utf-8")

# Verifica si el archivo demo_pathlib.txt existe y es un archivo regular
print(demo.exists(), demo.is_file())

# Lee el contenido del archivo demo_pathlib.txt usando UTF-8 como codificación y lo imprime
print(demo.read_text(encoding="utf-8"))

# Elimina el archivo demo_pathlib.txt, si existe, sin generar un error si no existe
demo.unlink(missing_ok=True)

True True
hola



Ejemplo 2. Listado de entradas con `Path.glob()` y `Path.iterdir()`. `glob()` permite filtrar por patron; `iterdir()` recorre todas las entradas y se puede combinar con `is_dir()` o `is_file()`.

In [67]:
from pathlib import Path

# Obtén el directorio actual como un objeto Path
base = Path.cwd()

# Encuentra todos los archivos con extensión .ipynb en el directorio actual
ipynb = list(base.glob("*.ipynb"))

# Encuentra todos los subdirectorios en el directorio actual y obtén sus nombres
carpetas = [p.name for p in base.iterdir() if p.is_dir()]

print(ipynb[:5])
print(carpetas[:5])

[WindowsPath('d:/recetas-python/Capitulo 5 (Archivos u I/cap5-1-7.ipynb'), WindowsPath('d:/recetas-python/Capitulo 5 (Archivos u I/cap5_part_02.ipynb')]
[]


Ejemplo 3. `os.scandir()` es util cuando se necesitan metadatos o rendimiento en directorios grandes. `itertools.batched()` permite trocear iterables en bloques sin cargar todo en memoria (se incluye una compatibilidad para Python < 3.12).

In [71]:
import itertools
import os

# Usa os.scandir para obtener una lista de los nombres de los archivos en el directorio actual
with os.scandir(".") as it:
    # Crea una lista de los nombres de las entradas que son archivos regulares
    entradas = [entry.name for entry in it if entry.is_file()]
print(entradas)


['cap5-1-7.ipynb', 'cap5_part_02.ipynb', 'data', 'data.bin', 'ejemplo.txt', 'ejemplo_pathlib.txt', 'newsample.bin', 'readme.txt', 'saludo.txt', 'saludo2.txt', 'sample.bin', 'somefile', 'somefile.bin', 'somefile.data', 'somefile.txt', 'somefile2.bin', 'somefile3', 'somefile4']


In [70]:

# Usa itertools.islice para leer el archivo somefile.data en bloques de 100 bytes
try:
    from itertools import batched
except ImportError:  # Python < 3.12
    
    # Define una función batched que divide un iterable en bloques de tamaño n
    def batched(iterable, n):
        """funcion que divide un iterable en bloques de tamaño n
        
        compatibilidad con python < 3.12

        """
        it = iter(iterable)
        while True:
            # Obtiene un bloque de tamaño n usando itertools.islice
            chunk = tuple(itertools.islice(it, n))
            if not chunk:
                break
            yield chunk

print(list(batched(range(10), 3)))

[(0, 1, 2), (3, 4, 5), (6, 7, 8), (9,)]
